In [1]:
from datasets import load_dataset
data = load_dataset("datadrivenscience/movie-genre-prediction")
data

/Users/thomaslazarus/.virtualenvs/notebooks/lib/python3.9/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm
Found cached dataset parquet (/Users/thomaslazarus/.cache/huggingface/datasets/datadrivenscience___parquet/datadrivenscience--movie-genre-prediction-01acd85570f2b187/0.0.0/2a3b91fbd88a2c90d1dbbb32b460cf621d31bd5b05b934492fdef7d8d6f236ec)
100%|██████████| 2/2 [00:00<00:00, 93.04it/s]


DatasetDict({
    train: Dataset({
        features: ['id', 'movie_name', 'synopsis', 'genre'],
        num_rows: 54000
    })
    test: Dataset({
        features: ['id', 'movie_name', 'synopsis', 'genre'],
        num_rows: 36000
    })
})

In [2]:
train_df = data["train"].to_pandas()
X_raw_train = train_df[["movie_name", "synopsis"]]
y_raw_train = train_df["genre"]
test_df = data["test"].to_pandas()
X_raw_test = test_df[["movie_name", "synopsis"]]
y_raw_test = test_df["genre"]
X_raw_train.head()

,movie_name,synopsis
0,Super Me,A young scriptwriter starts bringing valuable ...
1,Entity Project,A director and her friends renting a haunted h...
2,Behavioral Family Therapy for Serious Psychiat...,This is an educational video for families and ...
3,Blood Glacier,Scientists working in the Austrian Alps discov...
4,Apat na anino,Buy Day - Four Men Widely - Apart in Life - By...


In [3]:
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.preprocessing import MultiLabelBinarizer
from sklearn.multiclass import OneVsRestClassifier
from xgboost import XGBClassifier

print("Transforming y")
# Transform the multi-label genres into binary labels
mlb = MultiLabelBinarizer()
y_train_binary = mlb.fit_transform(y_raw_train)
y_test_binary = mlb.transform(y_raw_test)


print("Vecotrizing X")
vectorizer = CountVectorizer(stop_words='english')
X_train_transformed = vectorizer.fit_transform(X_raw_train)
X_test_transformed = vectorizer.transform(X_raw_test)

print("Fitting model")
# Train the XGBoost model using the binary relevance approach
xgb_model = OneVsRestClassifier(XGBClassifier(), n_jobs=5)
xgb_model.fit(X_train_transformed, y_train_binary)

Transforming y
Vecotrizing X
Fitting model


OneVsRestClassifier(estimator=XGBClassifier(base_score=None, booster=None,
                                            callbacks=None,
                                            colsample_bylevel=None,
                                            colsample_bynode=None,
                                            colsample_bytree=None,
                                            early_stopping_rounds=None,
                                            enable_categorical=False,
                                            eval_metric=None,
                                            feature_types=None, gamma=None,
                                            gpu_id=None, grow_policy=None,
                                            importance_type=None,
                                            interaction_constraints=None,
                                            learning_rate=None, max_bin=None,
                                            max_cat_threshold=None,
                                            max_cat_to_onehot=None,
                                            max_delta_step=None, max_depth=None,
                                            max_leaves=None,
                                            min_child_weight=None, missing=nan,
                                            monotone_constraints=None,
                                            n_estimators=100, n_jobs=None,
                                            num_parallel_tree=None,
                                            predictor=None, random_state=None, ...),
                    n_jobs=5)

In [4]:
# Evaluate the model
accuracy = xgb_model.score(X_test_transformed, y_test_binary)
print('Accuracy:', accuracy)

ValueError: Found input variables with inconsistent numbers of samples: [36000, 2]

In [9]:
X_test_transformed

(2, 2)

In [7]:
len(y_test_binary)

36000

In [10]:
X_raw_test

,movie_name,synopsis
0,A Death Sentence,"12 y.o. Ida's dad'll die without a DKK1,500,00..."
1,Intermedio,A group of four teenage friends become trapped...
2,30 Chua Phai Tet,A guy left his home for 12 years till he came ...
3,Paranoiac,A man long believed dead returns to the family...
4,Ordinary Happiness,"After a deadly accident, Paolo comes back on E..."
...,...,...
35995,Mysterious Doctor Satan,A mad scientist plots the domination of Americ...
35996,Blondes at Work,Steve promises his captain not to favor Torchy...
35997,The Hurricane,A Polynesian sailor unjustly imprisoned after ...
35998,Malayankunju,Anikkuttan is an ill-tempered electronics tech...


In [12]:
ddvectorizer.transform(X_raw_test[0])dd

KeyError: 0